<a href="https://colab.research.google.com/github/vishalgimhan/AI_Agents-Projects/blob/main/Multi_agent_Collaboration_Financial_Analysis_DeepLearningAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_huggingface langchain-core huggingface_hub langchain_groq

In [2]:
# Warnings Control
import warnings
warnings.filterwarnings('ignore')

In [14]:
from crewai import Agent, Task, Crew

In [4]:
import os
from google.colab import userdata

os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="meta-llama/llama-4-scout-17b-16e-instruct",
    api_key = userdata.get('GROQ_API_KEY')
)

## CrewAI Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Create Agents

In [8]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

In [9]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

In [11]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

In [12]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

## Creating Tasks

In [15]:
data_analysis_task = Task(
    description=("Continuously monitor and analyze market data for "
                "the selected stock ({stock_selection}). "
                "Use statistical modeling and machine learning to "
                "identify trends and predict market movements."),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [16]:
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [17]:
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [18]:
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)

In [20]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [21]:
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [22]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

Streaming output truncated to the last 5000 lines.

### Recommendations

- Monitor IV and HV closely to adjust trading strategies accordingly.
- Use technical indicators to identify entry and exit points.
- Maintain a disciplined risk management approach.

By following these insights and strategies, traders can navigate the risks associated with AAPL day trading effectively.

> Finished chain.
 

## Risk Assessment and Mitigation Strategies for AAPL Day Trading

### Historical Price Data Analysis

The historical price data for AAPL indicates significant price movements over the past year. With a current price of $202.82 USD, AAPL has shown volatility with daily price changes ranging from -0.22% to 1.83%. The historical prices over the past 6 months show:

- High: $257.83
- Low: $169.21
- Average: $220.45

This data suggests that AAPL has experienced considerable price swings, which is crucial for day traders to consider.

### Implied Volatility Analysis

The implied volatility (IV) for

In [23]:
from IPython.display import Markdown
Markdown(result)

{
"**Comprehensive Risk Analysis Report: AAPL Trading Strategy**": {

"**Executive Summary**": "This report evaluates the risks associated with the proposed trading strategies and execution plans for Apple Inc. (AAPL). The strategy involves buying AAPL when the price rebounds from the support zone near $193 and selling when the price approaches the resistance zone near $212.50. The analysis identifies potential risks, including market risk, position sizing risk, stop-loss risk, and diversification risk, as well as risks specific to AAPL stock such as regulatory risk, competition risk, supply chain risk, and technology risk. Mitigation strategies are recommended to manage these risks.",

"**Introduction**": "The proposed trading strategy for AAPL is based on technical analysis, focusing on support and resistance zones. The strategy includes position sizing of2-5% of the total portfolio, stop-loss orders2-3% below/above the entry price, and diversification. The current market conditions and historical data are crucial in understanding the potential risks and rewards of this strategy.",

"**Risks Associated with the Proposed Trading Strategy**": [
{
"**Risk**": "Market Risk",
"**Description**": "The strategy is based on technical analysis, and there is a risk that the support and resistance zones may not hold, leading to potential losses.",
"**Mitigation Strategy**": "Continuously monitor the stock's price movements and adjust the strategy as needed."
},
{
"**Risk**": "Position Sizing Risk",
"**Description**": "The position sizing of2-5% of the total portfolio may not be sufficient to manage risk, and a larger position size could lead to significant losses if the trade does not work out.",
"**Mitigation Strategy**": "Adjust position size based on risk tolerance and overall portfolio performance."
},
{
"**Risk**": "Stop-Loss Risk",
"**Description**": "The stop-loss orders may be triggered by short-term market volatility, leading to unnecessary losses.",
"**Mitigation Strategy**": "Use trailing stop-loss orders to protect profits and avoid unnecessary losses."
},
{
"**Risk**": "Diversification Risk",
"**Description**": "The strategy may not be diversified enough, as it focuses on a single stock, AAPL, which may increase the risk of losses.",
"**Mitigation Strategy**": "Diversify the portfolio to minimize risk and consider hedging strategies."
}
],

"**Risks Associated with AAPL Stock**": [
{
"**Risk**": "Regulatory Risk",
"**Description**": "AAPL is subject to various regulatory risks, including antitrust investigations and litigation, which could materially adversely affect the company's business, results of operations, and financial condition.",
"**Mitigation Strategy**": "Stay informed about regulatory changes and their potential impact on AAPL's business."
},
{
"**Risk**": "Competition Risk",
"**Description**": "AAPL faces significant competition in the global smartphone, personal computer, and tablet markets, which could affect its market share and profitability.",
"**Mitigation Strategy": "Monitor competitive developments and adjust the strategy accordingly."
},
{
"**Risk**": "Supply Chain Risk",
"**Description**": "AAPL relies on a complex global supply chain, which is subject to various risks, including natural disasters, information technology system failures, and commercial disputes.",
"**Mitigation Strategy**": "Diversify suppliers and maintain strategic partnerships. Implement contingency planning."
},
{
"**Risk**": "Technology Risk",
"**Description**": "AAPL's business and reputation are impacted by information technology system failures and network disruptions, which could materially adversely affect the company's business, results of operations, and financial condition.",
"**Mitigation Strategy**": "Implement robust IT infrastructure and cybersecurity measures to protect against technology risks."
}
],

"**Conclusion**": "The proposed trading strategy for AAPL involves various risks that need to be carefully managed. By continuously monitoring the stock's price movements, diversifying the portfolio, and staying informed about market trends, regulatory changes, and competitive developments, these risks can be mitigated. It is essential to implement a robust risk management plan to protect against potential losses and ensure the strategy's success."
}
}